In [ ]:
# Libraries (do not edit)
from ast import literal_eval

# Function to identify invalid email ID
def identify_invalid_email(record):
    # email = record.get('Email ID')
    email = record['Email ID']

     # Email ID can't be blank
    if len(email) == 0:
        record['Email ID'] = 'invalid'
        return record

    # Must contain '@' and '.'
    if '@' not in email or '.' not in email:
        record['Email ID'] = 'invalid'
        return record

    # '.' must come after '@' with at least one character in between
    at_index = email.find('@')
    dot_index = email.find('.')
    index_diff = dot_index - at_index
    if index_diff <= 1: record['Email ID'] = 'invalid'
    
    return record

# Inpout and output processing  (do not edit)
print(identify_invalid_email(literal_eval(input())))


#inputs:
'''
{'Candidate Name': 'Nikhil', 'Email ID': 'nikhil98singhdomain.com'}
{'Candidate Name': 'Dravid', 'Email ID': 'dravidlee@domaincom'}
{'Candidate Name': 'Rashid', 'Email ID': 'rashid.khan@domaincom'}
{'Candidate Name': 'Maggie', 'Email ID': '7777@domain.com'}
'''


In [ ]:
# Load the CSV file (do not edit)
import pandas as pd
df = pd.read_csv('https://d3ejq4mxgimsmf.cloudfront.net/AirPassengers-1eaa575774ad408691e22a1edfcab2ec.csv')

# Convert the 'Month' column to datetime format and extract the year from the 'Month' column
df['Year'] = pd.to_datetime(df['Month']).dt.year

# Group by year and calculate the mean passenger count
mean_passengers_by_year = df.groupby('Year')['Passengers'].mean()

# Input and output processing (do not edit)
mean_passengers_by_year = mean_passengers_by_year.round(0).astype(int)
print(mean_passengers_by_year.loc[int(input())])

In [ ]:
# Load the CSV file using Pandas (do not edit)
import pandas as pd
df = pd.read_csv('https://d3ejq4mxgimsmf.cloudfront.net/active_grandmasters_March25-b0e7a64fd25c48ab8f380debb40e8a0a.csv')

# Calculate the mean rating for each federation
federation_mean_stats = df.groupby('Fed')['Rating'].mean().reset_index()

# Add the list of players above the mean rating for each federation
federation_mean_stats['Players Above Mean'] = federation_mean_stats['Fed'].apply(
    lambda federation: df[(df['Fed'] == federation) & (df['Rating'] >= federation_mean_stats.loc[federation_mean_stats['Fed'] == federation, 'Rating'].values[0])]['Name'].tolist()
)

# Set the federation as the index
federation_mean_stats = federation_mean_stats.set_index('Fed')
players_above_mean = federation_mean_stats['Players Above Mean']

# Input and output processing (do not edit)
print(players_above_mean.loc[input()])

In [ ]:
import pandas as pd

input_data = input()
product_category, start_date, end_date = map(str.strip, input_data.split(','))

def aggregate_monthly_sales(input_tuple, filename='https://d3ejq4mxgimsmf.cloudfront.net/Product_sales_data-9f83ae7a11c340d1884ae214aadbacac.csv'):
    # Unpack the input tuple
    product_category, start_date, end_date = input_tuple

    # Read the CSV file
    df = pd.read_csv(filename)

    # Convert sale_date to datetime
    df['sale_date'] = pd.to_datetime(df['sale_date'])

    # Convert input dates to datetime
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    # Filter by product category and date range
    filtered_df = df[
        (df['product_category'] == product_category) &
        (df['sale_date'] >= start) &
        (df['sale_date'] <= end)
    ].copy()

    # Check if the date range spans more than one month
    if start.month != end.month:
        # Aggregate monthly
        filtered_df['month'] = filtered_df['sale_date'].dt.to_period('M').astype(str)
        result = filtered_df.groupby('month')['sales_amount'].sum().reset_index()
        return list(result.itertuples(index=False, name=None))
    else:
        # Aggregate daily
        filtered_df.loc[:, 'day'] = filtered_df['sale_date'].dt.to_period('D').astype(str)
        result = filtered_df.groupby('day')['sales_amount'].sum().reset_index()
        return list(result.itertuples(index=False, name=None))

output = aggregate_monthly_sales((product_category, start_date, end_date))
print(output)


In [ ]:
# Loading the data (do not edit)
import pandas as pd
filename = 'https://d3ejq4mxgimsmf.cloudfront.net/hourly_energy_usage-9eb4bd3748964f8da8d95b54df732b1d.csv'
df = pd.read_csv(filename)

def analyze_household_peak_usage(household_id):
    # Define the segments and their corresponding hour indices
    segments = {
        'Late Night/Early Morning': list(range(0, 6)),
        'Morning': list(range(6, 12)),
        'Afternoon': list(range(12, 18)),
        'Evening/Night': list(range(18, 24))
    }

    # Filter the row for the specified household
    household_data = df[df['Household_ID'] == household_id]

    # Extract hourly values
    hourly_values = household_data.iloc[0, 1:].values  # Skip Household_ID column

    # Calculate average consumption for each segment
    segment_averages = {}
    for segment_name, indices in segments.items():
        segment_values = [hourly_values[i] for i in indices]  # The indices here accurately match the hour columns
        segment_avg = sum(segment_values) / len(segment_values)
        segment_averages[segment_name] = segment_avg

    # Identify the segment with the highest average consumption
    peak_segment = max(segment_averages, key = segment_averages.get)
    peak_indices = segments[peak_segment]
    peak_avg = segment_averages[peak_segment]

    # Find hours in the peak segment where consumption is above the segment average
    high_usage_hours = [i for i in peak_indices if hourly_values[i] > peak_avg]

    # Return the result as a dictionary
    return {'Peak Segment': peak_segment, 'High Usage Hours': high_usage_hours}

# Input and output processing (do not edit)
print(analyze_household_peak_usage(input()))